# Import Modules and Data

In [63]:
# Import Modules
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image


In [64]:
# Import the pitcher data
verlander_df = pd.read_csv(Path("../resources/verlander_update.csv"))

display(verlander_df.head())

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,stand,...,on_1b,outs_when_up,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,9/16/2022,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,5,Slider,0,2,Infield shift,Standard
1,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard
2,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,3,Slider,0,2,Infield shift,Standard
3,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard
4,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,...,NaN,2,5,35,1,Slider,0,2,Infield shift,Standard


In [65]:
# Create a pitch count column
verlander_df['ones'] = 1

pitch_count_df = verlander_df[['game_date', 'ones']]
pitch_count_df['pitch_count'] = pitch_count_df.groupby(['game_date']).cumcount(ascending = False)
pitch_count_df['pitch_count'] = pitch_count_df['pitch_count'] + 1
pitch_count_df

c:\Users\britt\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\britt\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,game_date,ones,pitch_count
0,9/16/2022,1,79
1,9/16/2022,1,78
2,9/16/2022,1,77
3,9/16/2022,1,76
4,9/16/2022,1,75
...,...,...,...
2331,4/9/2022,1,5
2332,4/9/2022,1,4
2333,4/9/2022,1,3
2334,4/9/2022,1,2


In [66]:
# Add the pitch count column to the original verlander dataframe
verlander_df = pd.concat([verlander_df, pitch_count_df['pitch_count']], axis = 1)
verlander_df.head()

,pitch_type,game_date,player_name,batter,pitcher,events,description,zone,des,stand,...,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,ones,pitch_count
0,SL,9/16/2022,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,...,5,35,5,Slider,0,2,Infield shift,Standard,1,79
1,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,...,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard,1,78
2,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,...,5,35,3,Slider,0,2,Infield shift,Standard,1,77
3,FF,9/16/2022,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,...,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard,1,76
4,SL,9/16/2022,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,...,5,35,1,Slider,0,2,Infield shift,Standard,1,75


# Clean the Data

In [67]:
# Drop extra columns
verlander_df = verlander_df.drop(columns=['player_name', 'batter', 'pitcher', 'events', 'des', 'home_team', 'away_team', 'hit_location', 'bb_type', 'at_bat_number', 'ones', 'p_throws', 'description'])

display(verlander_df.head())

,pitch_type,game_date,zone,stand,type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,pitch_count
0,SL,9/16/2022,9,R,S,1,2,NaN,NaN,NaN,2,5,5,Slider,0,2,Infield shift,Standard,79
1,FF,9/16/2022,3,R,S,1,2,NaN,NaN,NaN,2,5,4,4-Seam Fastball,0,2,Infield shift,Standard,78
2,SL,9/16/2022,13,R,S,1,1,NaN,NaN,NaN,2,5,3,Slider,0,2,Infield shift,Standard,77
3,FF,9/16/2022,1,R,S,1,0,NaN,NaN,NaN,2,5,2,4-Seam Fastball,0,2,Infield shift,Standard,76
4,SL,9/16/2022,8,R,B,0,0,NaN,NaN,NaN,2,5,1,Slider,0,2,Infield shift,Standard,75


In [68]:
# Change all NaN in the on base columns to 0.0
on_base_df = verlander_df[['on_3b', 'on_2b', 'on_1b']]
on_base_df = on_base_df.fillna(0)
on_base_df.head(25)

,on_3b,on_2b,on_1b
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


In [69]:
# Change player ID numbers for the on base dataframe to 1.0
on_base_df.on_1b[on_base_df.on_1b > 0] = 1.0
on_base_df.on_2b[on_base_df.on_2b > 0] = 1.0
on_base_df.on_3b[on_base_df.on_3b > 0] = 1.0

on_base_df[225:255]

,on_3b,on_2b,on_1b
225,0.0,0.0,0.0
226,0.0,0.0,0.0
227,0.0,0.0,0.0
228,0.0,0.0,0.0
229,0.0,0.0,0.0
230,1.0,0.0,0.0
231,1.0,0.0,0.0
232,1.0,0.0,0.0
233,1.0,0.0,0.0
234,1.0,0.0,0.0


In [70]:
# Drop the original on base columns 
verlander_df = verlander_df.drop(columns = ['on_3b', 'on_2b', 'on_1b'])
verlander_df.head()

,pitch_type,game_date,zone,stand,type,balls,strikes,outs_when_up,inning,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,pitch_count
0,SL,9/16/2022,9,R,S,1,2,2,5,5,Slider,0,2,Infield shift,Standard,79
1,FF,9/16/2022,3,R,S,1,2,2,5,4,4-Seam Fastball,0,2,Infield shift,Standard,78
2,SL,9/16/2022,13,R,S,1,1,2,5,3,Slider,0,2,Infield shift,Standard,77
3,FF,9/16/2022,1,R,S,1,0,2,5,2,4-Seam Fastball,0,2,Infield shift,Standard,76
4,SL,9/16/2022,8,R,B,0,0,2,5,1,Slider,0,2,Infield shift,Standard,75


In [71]:
# Concat the new on base columns to the verlander df
verlander_df = pd.concat([verlander_df, on_base_df], axis = 1)

verlander_df[225: 255]

,pitch_type,game_date,zone,stand,type,balls,strikes,outs_when_up,inning,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,pitch_count,on_3b,on_2b,on_1b
225,FF,8/23/2022,13,R,S,1,0,1,1,2,4-Seam Fastball,0,0,Standard,Standard,5,0.0,0.0,0.0
226,CU,8/23/2022,14,R,B,0,0,1,1,1,Curveball,0,0,Standard,Standard,4,0.0,0.0,0.0
227,SL,8/23/2022,3,L,X,0,2,0,1,3,Slider,0,0,Standard,Standard,3,0.0,0.0,0.0
228,CU,8/23/2022,8,L,S,0,1,0,1,2,Curveball,0,0,Standard,Standard,2,0.0,0.0,0.0
229,FF,8/23/2022,8,L,S,0,0,0,1,1,4-Seam Fastball,0,0,Standard,Standard,1,0.0,0.0,0.0
230,SL,8/16/2022,14,R,X,0,2,2,7,4,Slider,3,3,Standard,Standard,103,1.0,0.0,0.0
231,FF,8/16/2022,11,R,S,0,2,2,7,3,4-Seam Fastball,3,3,Standard,Standard,102,1.0,0.0,0.0
232,FF,8/16/2022,11,R,S,0,1,2,7,2,4-Seam Fastball,3,3,Standard,Standard,101,1.0,0.0,0.0
233,SL,8/16/2022,12,R,S,0,0,2,7,1,Slider,3,3,Standard,Standard,100,1.0,0.0,0.0
234,FF,8/16/2022,11,R,S,2,2,1,7,5,4-Seam Fastball,3,3,Infield shift,Standard,99,1.0,0.0,0.0


In [72]:
# Converting batting score and fielding score to one column.
# Positive number means fielding team is winning and negative number means batting team is winning. 
verlander_df['score_diff'] = verlander_df['fld_score'] - verlander_df['bat_score']


# Drop batting score and fielding score columns now that you have the score differential
verlander_df = verlander_df.drop(columns = ['bat_score', 'fld_score'])

verlander_df.head()

,pitch_type,game_date,zone,stand,type,balls,strikes,outs_when_up,inning,pitch_number,pitch_name,if_fielding_alignment,of_fielding_alignment,pitch_count,on_3b,on_2b,on_1b,score_diff
0,SL,9/16/2022,9,R,S,1,2,2,5,5,Slider,Infield shift,Standard,79,0.0,0.0,0.0,2
1,FF,9/16/2022,3,R,S,1,2,2,5,4,4-Seam Fastball,Infield shift,Standard,78,0.0,0.0,0.0,2
2,SL,9/16/2022,13,R,S,1,1,2,5,3,Slider,Infield shift,Standard,77,0.0,0.0,0.0,2
3,FF,9/16/2022,1,R,S,1,0,2,5,2,4-Seam Fastball,Infield shift,Standard,76,0.0,0.0,0.0,2
4,SL,9/16/2022,8,R,B,0,0,2,5,1,Slider,Infield shift,Standard,75,0.0,0.0,0.0,2


In [73]:
# Convert balls column and strikes column to one column that indicates whether or not it's a hitter's count, a pitcher's count, or neutral.

def count(row):
    if (row['balls'] == 0 and row['strikes'] == 0) or (row['balls'] == 1 and row['strikes'] == 0) or (row['balls'] == 2 and row['strikes'] == 0) or (row['balls'] == 2 and row['strikes'] == 1) or (row['balls'] == 3 and row['strikes'] == 0) or (row['balls'] == 3 and row['strikes'] == 1):
        val = 'H'
    elif (row['balls'] == 0 and row['strikes'] == 1) or (row['balls'] == 0 and row['strikes'] == 2) or (row['balls'] == 1 and row['strikes'] == 2) or (row['balls'] == 2 and row['strikes'] == 2):
        val = 'P'
    else:
        val = 'N'
    return val

verlander_df['count_type'] = verlander_df.apply(count, axis = 1)


verlander_df.head()

,pitch_type,game_date,zone,stand,type,balls,strikes,outs_when_up,inning,pitch_number,pitch_name,if_fielding_alignment,of_fielding_alignment,pitch_count,on_3b,on_2b,on_1b,score_diff,count_type
0,SL,9/16/2022,9,R,S,1,2,2,5,5,Slider,Infield shift,Standard,79,0.0,0.0,0.0,2,P
1,FF,9/16/2022,3,R,S,1,2,2,5,4,4-Seam Fastball,Infield shift,Standard,78,0.0,0.0,0.0,2,P
2,SL,9/16/2022,13,R,S,1,1,2,5,3,Slider,Infield shift,Standard,77,0.0,0.0,0.0,2,N
3,FF,9/16/2022,1,R,S,1,0,2,5,2,4-Seam Fastball,Infield shift,Standard,76,0.0,0.0,0.0,2,H
4,SL,9/16/2022,8,R,B,0,0,2,5,1,Slider,Infield shift,Standard,75,0.0,0.0,0.0,2,H


In [74]:
# Combine the ball and strikes column into one column and then drop the individual columns.
verlander_df['count'] = verlander_df['balls'].astype(str) +'-'+ verlander_df['strikes'].astype(str)
verlander_df = verlander_df.drop(columns = ['balls', 'strikes'])
verlander_df.head()

,pitch_type,game_date,zone,stand,type,outs_when_up,inning,pitch_number,pitch_name,if_fielding_alignment,of_fielding_alignment,pitch_count,on_3b,on_2b,on_1b,score_diff,count_type,count
0,SL,9/16/2022,9,R,S,2,5,5,Slider,Infield shift,Standard,79,0.0,0.0,0.0,2,P,1-2
1,FF,9/16/2022,3,R,S,2,5,4,4-Seam Fastball,Infield shift,Standard,78,0.0,0.0,0.0,2,P,1-2
2,SL,9/16/2022,13,R,S,2,5,3,Slider,Infield shift,Standard,77,0.0,0.0,0.0,2,N,1-1
3,FF,9/16/2022,1,R,S,2,5,2,4-Seam Fastball,Infield shift,Standard,76,0.0,0.0,0.0,2,H,1-0
4,SL,9/16/2022,8,R,B,2,5,1,Slider,Infield shift,Standard,75,0.0,0.0,0.0,2,H,0-0


In [75]:
# Check what columns are in the dataframe.
verlander_df.columns

Index(['pitch_type', 'game_date', 'zone', 'stand', 'type', 'outs_when_up',
       'inning', 'pitch_number', 'pitch_name', 'if_fielding_alignment',
       'of_fielding_alignment', 'pitch_count', 'on_3b', 'on_2b', 'on_1b',
       'score_diff', 'count_type', 'count'],
      dtype='object')

In [76]:
# Shift columns to inform previous pitch.
lagged_df = verlander_df[['zone', 'type', 'pitch_name']].shift(-1)
lagged_df

,zone,type,pitch_name
0,3.0,S,4-Seam Fastball
1,13.0,S,Slider
2,1.0,S,4-Seam Fastball
3,8.0,B,Slider
4,4.0,X,Slider
...,...,...,...
2331,4.0,S,4-Seam Fastball
2332,13.0,B,4-Seam Fastball
2333,13.0,S,4-Seam Fastball
2334,11.0,B,4-Seam Fastball


In [77]:
# Rename the lagged columns to have 'previous' indicator.
lagged_df = lagged_df.rename(columns = {
    'zone' : 'previous_zone',
    'type' : 'previous_type',
    'pitch_name' : 'previous_pitch'
})

# Drop NaN columns
lagged_df = lagged_df.dropna()
lagged_df

,previous_zone,previous_type,previous_pitch
0,3.0,S,4-Seam Fastball
1,13.0,S,Slider
2,1.0,S,4-Seam Fastball
3,8.0,B,Slider
4,4.0,X,Slider
...,...,...,...
2330,8.0,S,4-Seam Fastball
2331,4.0,S,4-Seam Fastball
2332,13.0,B,4-Seam Fastball
2333,13.0,S,4-Seam Fastball


In [78]:
# Concat the previous columns on to the dataframe and drop the original columns.
verlander_lagged_df = pd.concat([verlander_df, lagged_df], join='inner', axis = 1)

verlander_lagged_df = verlander_lagged_df.drop(columns = ['zone', 'type', 'pitch_name'])

verlander_lagged_df

,pitch_type,game_date,stand,outs_when_up,inning,pitch_number,if_fielding_alignment,of_fielding_alignment,pitch_count,on_3b,on_2b,on_1b,score_diff,count_type,count,previous_zone,previous_type,previous_pitch
0,SL,9/16/2022,R,2,5,5,Infield shift,Standard,79,0.0,0.0,0.0,2,P,1-2,3.0,S,4-Seam Fastball
1,FF,9/16/2022,R,2,5,4,Infield shift,Standard,78,0.0,0.0,0.0,2,P,1-2,13.0,S,Slider
2,SL,9/16/2022,R,2,5,3,Infield shift,Standard,77,0.0,0.0,0.0,2,N,1-1,1.0,S,4-Seam Fastball
3,FF,9/16/2022,R,2,5,2,Infield shift,Standard,76,0.0,0.0,0.0,2,H,1-0,8.0,B,Slider
4,SL,9/16/2022,R,2,5,1,Infield shift,Standard,75,0.0,0.0,0.0,2,H,0-0,4.0,X,Slider
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2330,FF,4/9/2022,R,1,1,1,Infield shift,Standard,6,0.0,0.0,0.0,0,H,0-0,8.0,S,4-Seam Fastball
2331,FF,4/9/2022,L,0,1,5,Infield shift,Standard,5,0.0,0.0,0.0,0,P,2-2,4.0,S,4-Seam Fastball
2332,FF,4/9/2022,L,0,1,4,Infield shift,Standard,4,0.0,0.0,0.0,0,H,2-1,13.0,B,4-Seam Fastball
2333,FF,4/9/2022,L,0,1,3,Infield shift,Standard,3,0.0,0.0,0.0,0,N,1-1,13.0,S,4-Seam Fastball


In [79]:
# Recheck that you have the columns you want. 
verlander_lagged_df.columns

Index(['pitch_type', 'game_date', 'stand', 'outs_when_up', 'inning',
       'pitch_number', 'if_fielding_alignment', 'of_fielding_alignment',
       'pitch_count', 'on_3b', 'on_2b', 'on_1b', 'score_diff', 'count_type',
       'count', 'previous_zone', 'previous_type', 'previous_pitch'],
      dtype='object')

# Configuring the data for the model(s)

In [80]:
# Split target columns from features
y = verlander_lagged_df['pitch_type']
X = verlander_lagged_df.drop(columns = 'pitch_type')

In [81]:
# See which variables are categorical 
X.dtypes

game_date                 object
stand                     object
outs_when_up               int64
inning                     int64
pitch_number               int64
if_fielding_alignment     object
of_fielding_alignment     object
pitch_count                int64
on_3b                    float64
on_2b                    float64
on_1b                    float64
score_diff                 int64
count_type                object
count                     object
previous_zone            float64
previous_type             object
previous_pitch            object
dtype: object

In [82]:
# Convert previous_zone to object because it is a category
X['previous_zone'] = X['previous_zone'].astype(str)

# Create a list of categorical variables
categorical_variables = list(X.dtypes[X.dtypes == 'object'].index)
categorical_variables.remove('game_date')

# Display the categorical variables list
categorical_variables

['stand',
 'if_fielding_alignment',
 'of_fielding_alignment',
 'count_type',
 'count',
 'previous_zone',
 'previous_type',
 'previous_pitch']

In [83]:
# Change categorical variables using OneHotEncoder
enc = OneHotEncoder(sparse = False)

encoded_data = enc.fit_transform(X[categorical_variables])

encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

numerical_variables = X.drop(columns = categorical_variables)

X_encoded_df = pd.concat([encoded_df, numerical_variables], axis = 1)

X_encoded_df.head()

c:\Users\britt\anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,stand_L,stand_R,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Standard,of_fielding_alignment_Strategic,count_type_H,count_type_N,count_type_P,...,previous_pitch_Slider,game_date,outs_when_up,inning,pitch_number,pitch_count,on_3b,on_2b,on_1b,score_diff
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,9/16/2022,2,5,5,79,0.0,0.0,0.0,2
1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,9/16/2022,2,5,4,78,0.0,0.0,0.0,2
2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,9/16/2022,2,5,3,77,0.0,0.0,0.0,2
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,9/16/2022,2,5,2,76,0.0,0.0,0.0,2
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,9/16/2022,2,5,1,75,0.0,0.0,0.0,2


In [84]:
# Drop game date as it is no longer necessary
X_encoded_df = X_encoded_df.drop(columns = 'game_date')
X_encoded_df.columns

Index(['stand_L', 'stand_R', 'if_fielding_alignment_Infield shift',
       'if_fielding_alignment_Standard', 'if_fielding_alignment_Strategic',
       'of_fielding_alignment_Standard', 'of_fielding_alignment_Strategic',
       'count_type_H', 'count_type_N', 'count_type_P', 'count_0-0',
       'count_0-1', 'count_0-2', 'count_1-0', 'count_1-1', 'count_1-2',
       'count_2-0', 'count_2-1', 'count_2-2', 'count_3-0', 'count_3-1',
       'count_3-2', 'previous_zone_1.0', 'previous_zone_11.0',
       'previous_zone_12.0', 'previous_zone_13.0', 'previous_zone_14.0',
       'previous_zone_2.0', 'previous_zone_3.0', 'previous_zone_4.0',
       'previous_zone_5.0', 'previous_zone_6.0', 'previous_zone_7.0',
       'previous_zone_8.0', 'previous_zone_9.0', 'previous_type_B',
       'previous_type_S', 'previous_type_X', 'previous_pitch_4-Seam Fastball',
       'previous_pitch_Changeup', 'previous_pitch_Curveball',
       'previous_pitch_Slider', 'outs_when_up', 'inning', 'pitch_number',
       'p

In [85]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, random_state=1)

In [86]:
# Scaling Data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Decision Tree Model

In [87]:
# Creating the Decision Tree Model
dt_model = tree.DecisionTreeClassifier()
dt_model = dt_model.fit(X_train_scaled, y_train)
predictions = dt_model.predict(X_test_scaled)

In [88]:
# Displaying results
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

          CH       0.00      0.00      0.00         9
          CU       0.29      0.28      0.29       113
          FF       0.53      0.53      0.53       277
          SL       0.41      0.41      0.41       185

    accuracy                           0.44       584
   macro avg       0.31      0.31      0.31       584
weighted avg       0.44      0.44      0.44       584



In [89]:
# Create Decision Tree Graph
dot_data = tree.export_graphviz(
    dt_model, out_file=None, feature_names = X_encoded_df.columns, class_names = ['0', '1', '2', '3'], filled = True
)

graph = pydotplus.graph_from_dot_data(dot_data)

Image(graph.create_png())

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.820549 to fit



# Random Forest Model

In [90]:
# Creating the random forest model
rf_model = RandomForestClassifier(n_estimators = 1000, random_state = 1)
rf_model = rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)

In [91]:
# Displaying results
print(classification_report(y_test, rf_predictions))

              precision    recall  f1-score   support

          CH       0.00      0.00      0.00         9
          CU       0.39      0.15      0.22       113
          FF       0.54      0.73      0.62       277
          SL       0.46      0.41      0.43       185

    accuracy                           0.51       584
   macro avg       0.35      0.32      0.32       584
weighted avg       0.48      0.51      0.47       584



# Feature Importance

In [92]:
# Zip the feature importances with the associated feature name
importances = rf_model.feature_importances_
important_features = zip(X_encoded_df.columns, importances)
important_features

In [93]:
# Calculating the most important features that the Random Forest model used
importances_df = pd.DataFrame(important_features)
importances_df = importances_df.rename(columns = {0: 'Feature', 1: 'Importance'})
importances_df = importances_df.set_index('Feature')
importances_df = importances_df.sort_values(by = 'Importance', ascending = False)
importances_df

,Importance
Feature,
pitch_count,0.158612
inning,0.089474
score_diff,0.087982
outs_when_up,0.060577
pitch_number,0.048294
on_1b,0.032687
previous_pitch_4-Seam Fastball,0.024529
if_fielding_alignment_Infield shift,0.022926
if_fielding_alignment_Standard,0.022371
